In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/Major\ Project

/content/drive/MyDrive/Major Project


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
import os
import numpy as np
from matplotlib import pyplot as plt
import copy
import PIL
from torchvision import models
from PIL import Image
import pandas as pd

In [5]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
# Data loading (Modiflication required)
from torchvision import transforms,datasets
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
data_dir = 'ImageDatasets/BnWDataset/'
image_dataset = datasets.ImageFolder(data_dir,transform)
total_count = len(image_dataset)
train_count = int(0.8 * total_count)
test_count = total_count - train_count
train_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_count, test_count])

dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                             shuffle=True, num_workers=4)}
dataset_sizes = {'train': len(train_dataset),'val':len(test_dataset)}
class_names = image_dataset.classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
# Downloading VGG network
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)
print(vgg16)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [11]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        vgg16=models.vgg16(pretrained=True)
        Children = list(vgg16.children())
        Features = list(Children[0].children())
        FeaturesD = []
        FeaturesD.extend(Features[19:21])
        FeaturesD.append(nn.Dropout2d(p=0.4, inplace=False))
        FeaturesD.extend(Features[21:24])
        FeaturesD.append(nn.Dropout2d(p=0.4, inplace=False))
        FeaturesD.extend(Features[24:26])
        FeaturesD.append(nn.Dropout2d(p=0.4, inplace=False))
        FeaturesD.extend(Features[26:])
                     
        #Features[:-10]
        #Children[1]
        self.features1 = nn.Sequential(*Features[0:19])
        self.features2 = nn.Sequential(*FeaturesD)#nn.Sequential(*Features[19:])
        self.pool = nn.Sequential(Children[1])
        self.classifier = nn.Sequential(
                      nn.Linear(in_features=25088, out_features=4096, bias=True),
                      nn.ReLU(inplace=True),
                      nn.Dropout(p=0.5, inplace=False),
                      nn.Linear(in_features=4096, out_features=4096, bias=True),
                      nn.ReLU(inplace=True),
                      nn.Dropout(p=0.5, inplace=False),
                      nn.Linear(in_features=4096, out_features=len(class_names), bias=True)
                  )
    def forward(self,x):
        x = self.features1(x)
        x = self.features2(x)
        x = self.pool(x)
        x = x.view(-1,25088)
        x = self.classifier(x)
        return x

In [12]:
Net = Network()
#Net

In [14]:
for name, p in Net.named_parameters():
        if "Features1" in name:
            p.requires_grad = False

In [15]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    try:
      for epoch in range(num_epochs):
        
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          for phase in ['train', 'val']:
              if phase == 'train':
                  scheduler.step()
                  model.train()  # Set model to training mode
              else:
                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data.
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  optimizer.zero_grad()

                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      outputs = model(inputs)
                      _, preds = torch.max(outputs, 1)
                      loss = criterion(outputs, labels)

                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
              print(f"{phase} Running Corrects:,{running_corrects.double()}")
              print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                  phase, epoch_loss, epoch_acc))

              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
        
                  best_model_wts = copy.deepcopy(model.state_dict())
          print()
    except KeyboardInterrupt:
      print('Interrupting Training...')
      print()
      model.load_state_dict(best_model_wts)  
      if epoch<1:
        return model    

    model.load_state_dict(best_model_wts)  
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

In [16]:
def ConfMat(model,dt = 'val'):
  since=time.time()
  confusionMatrix=np.zeros((len(class_names),len(class_names)),dtype=int)
  with torch.no_grad():
      for data in dataloaders[dt]:
          images, labels = data
          images=images.to(device)
          labels=labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          for i in range(4):
            try:
              confusionMatrix[labels[i],predicted[i]]+=1
            except:
              continue
  confusionMatrix=pd.DataFrame(confusionMatrix,columns=['Pred '+class_names[i] for i in range(len(class_names))],index=['Actual '+class_names[i] for i in range(len(class_names))])
  time_elapsed = time.time() - since
  print('Validation completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  return confusionMatrix

In [17]:
import torch.optim as optim
from torch.optim import lr_scheduler


Net = Net.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)
#optimizer_ft = optim.SGD(Net.parameters(), lr=0.005, momentum=0.9)
#optimizer_ft = optim.Adam(Net.parameters(),lr=0.0005)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [18]:
try:
  del model_ft
except:
  pass

In [19]:
model_ft = train_model(Net, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

train Running Corrects:,2751.0
train Loss: 1.9694 Acc: 0.3034
val Running Corrects:,1100.0
val Loss: 1.4054 Acc: 0.4852

Epoch 1/24
----------
train Running Corrects:,4678.0
train Loss: 1.3242 Acc: 0.5159
val Running Corrects:,1318.0
val Loss: 1.1526 Acc: 0.5814

Epoch 2/24
----------
train Running Corrects:,5230.0
train Loss: 1.1431 Acc: 0.5768
val Running Corrects:,1404.0
val Loss: 0.9909 Acc: 0.6193

Epoch 3/24
----------
train Running Corrects:,5661.0
train Loss: 1.0258 Acc: 0.6243
val Running Corrects:,1486.0
val Loss: 0.8929 Acc: 0.6555

Epoch 4/24
----------
train Running Corrects:,5894.0
train Loss: 0.9463 Acc: 0.6500
val Running Corrects:,1568.0
val Loss: 0.8862 Acc: 0.6917

Epoch 5/24
----------
train Running Corrects:,6171.0
train Loss: 0.8648 Acc: 0.6805
val Running Corrects:,1530.0
val Loss: 0.9197 Acc: 0.6749

Epoch 6/24
----------
train Running Corrects:,6927.0
train Loss: 0.6441 Acc: 0.7639
val Running Corrects:,1637.0
val Loss: 0.7710 Acc: 0.7221

Epoch 7/24
----------

In [15]:
torch.save(model_ft.state_dict(),'CNN/VGG16_half_freezeBNW2.pt')

In [16]:
ConfMat(model_ft)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation completed in 1m 8s


,Pred female_angry,Pred female_disgust,Pred female_fear,Pred female_happy,Pred female_neutral,Pred female_sad,Pred female_surprise,Pred male_angry,Pred male_fear,Pred male_happy,Pred male_neutral,Pred male_sad,Pred male_surprise
Actual female_angry,185,7,1,8,4,1,2,6,1,0,0,0,0
Actual female_disgust,16,141,6,6,7,13,0,0,2,0,0,2,0
Actual female_fear,6,5,170,12,5,34,0,0,0,0,0,0,0
Actual female_happy,12,11,12,166,5,4,1,0,0,1,1,0,0
Actual female_neutral,4,8,1,3,171,9,0,0,0,1,0,0,0
Actual female_sad,2,17,20,0,17,169,0,0,0,0,0,6,0
Actual female_surprise,0,1,0,1,0,0,115,0,0,0,0,0,0
Actual male_angry,4,0,0,0,1,0,0,135,5,20,9,6,1
Actual male_fear,0,1,4,0,0,1,0,13,70,20,7,34,3
Actual male_happy,2,0,1,3,0,0,0,23,21,94,5,5,4


In [17]:
ConfMat(model_ft,'train')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation completed in 4m 24s


,Pred female_angry,Pred female_disgust,Pred female_fear,Pred female_happy,Pred female_neutral,Pred female_sad,Pred female_surprise,Pred male_angry,Pred male_fear,Pred male_happy,Pred male_neutral,Pred male_sad,Pred male_surprise
Actual female_angry,868,8,0,4,1,0,0,0,0,0,0,0,0
Actual female_disgust,11,865,1,0,3,23,0,0,0,0,0,0,0
Actual female_fear,2,0,829,4,0,29,0,0,0,0,0,0,0
Actual female_happy,7,1,0,871,2,2,0,0,0,0,0,0,0
Actual female_neutral,0,4,0,0,851,4,0,0,0,0,0,0,0
Actual female_sad,0,10,17,0,9,829,0,0,0,0,0,0,0
Actual female_surprise,0,0,0,0,0,0,379,0,0,0,0,0,0
Actual male_angry,0,0,0,0,0,0,0,630,4,7,3,2,0
Actual male_fear,0,0,0,0,0,0,0,7,578,15,4,68,2
Actual male_happy,0,1,0,2,0,0,0,9,20,619,9,3,6


In [ ]:
model = Network()
model.load_state_dict(torch.load('CNN/VGG16_half_freeze.pt',map_location=torch.device(device)))
model = model.to(device)
model.eval();